<a href="https://colab.research.google.com/github/GinkGoPi/researches/blob/main/token-stake/booster.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Staking with lock to booster

通过质押token并且锁仓一定时间，可以获取boost的收益

奖励部分的逻辑与`SimepleStakingRewards`一致，只是其中用户质押的数据需要按照`BoostPool`中的**份额**来替代计算

核心的问题：
- 质押的数量转换成份额数
- **Lock**周期带来的Boost收益系数


In [33]:
import calendar
import time
import datetime
from datetime import datetime
from datetime import timedelta

In [2]:
calendar.timegm(time.gmtime())

1675305783

In [49]:
class StakingRewards(object):
    
    def __init__(self) -> None:
        self.stakingToken = 'stakingToken'
        self.rewardsToken = 'rewardsToken'

        self.periodFinish = 0
        self.lastUpdateTime = 0
        self.duration = 1 * 60*60*24 # seconds

        self.rewardRate = 0
        self.rewardPerTokenStored = 0

        self.userRewardPerTokenPaid = {'boostPool': 0, 'alice': 0, 'bob': 0, 'kite': 0}
        self.rewards = {'boostPool': 0, 'alice': 0, 'bob': 0, 'kite': 0}

        # Note: this data will be from `BoostPool`
        # self.totalSupply = 0
        # self.balanceOf = {'boostPool': 0, 'alice': 0, 'bob': 0, 'kite': 0}
    
    # def stake(self, address: str, amount: int):
    #     self.updateReward(address)
    #     self.balanceOf[address] += amount
    #     self.totalSupply += amount
    
    # def withdraw(self, to: str, amount: int):
    #     self.updateReward(to)
    #     self.balanceOf[to] -= amount
    #     self.totalSupply -= amount
    #     return amount
      
    def updateReward(self, _account: str):
        self.rewardPerTokenStored = self.rewardPerToken()
        self.lastUpdateTime = self.lastTimeRewardApplicable()

        if _account != "address_0":
            self.rewards[_account] = self.earned(_account)
            self.userRewardPerTokenPaid[_account] = self.rewardPerTokenStored
    
    def rewardPerToken(self):
        if self.totalSupply == 0:
            return self.rewardPerTokenStored
        else:
            return self.rewardPerTokenStored + (self.rewardRate * (self.lastTimeRewardApplicable() - self.lastUpdateTime) * 1e18) / self.totalSupply

    def earned(self, _account: str):
        return ((self.balanceOf[_account] *
                (self.rewardPerToken() - self.userRewardPerTokenPaid[_account])) / 1e18) + self.rewards[_account]

    def getReward(self, to: str):
        """Gain reward token"""
        self.updateReward(to)
        reward = self.rewards[to]
        if reward > 0:
            self.rewards[to] = 0
            # rewardsToken.transfer(to, reward)
        
        return reward

    def notifyRewardAmount(self, _amount: int):
        self.updateReward("address_0")
        if self.blockTimestamp() >= self.periodFinish:
            self.rewardRate = _amount / self.duration
        else:
            remainingRewards = (self.periodFinish - self.blockTimestamp()) * self.rewardRate
            self.rewardRate = (_amount + remainingRewards) / self.duration
        

        assert(self.rewardRate > 0, "reward rate = 0")

        self.lastUpdateTime = self.blockTimestamp()
        self.periodFinish = self.blockTimestamp() + self.duration
        
    
    def lastTimeRewardApplicable(self):
        return self.periodFinish if self.periodFinish <= self.blockTimestamp() else self.blockTimestamp()

    def blockTimestamp(self):
        """Akin to `block.timestamp`"""
        return calendar.timegm(time.gmtime())

<>:69: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:69: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<ipython-input-49-c96011e66ff7>:69: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(self.rewardRate > 0, "reward rate = 0")


In [50]:
class UserInfo(object):
    def __init__(self, 
        shares=0, 
        lastDepositedTime=0, 
        tokenAtLastUserAction=0, 
        lastUserActionTime=0,
        lockStartTime=0,
        lockEndTime=0,
        userBoostedShare=0,
        locked=False,
        lockedAmount=0):
      self.shares = shares
      self.lastDepositedTime = lastDepositedTime
      self.tokenAtLastUserAction = tokenAtLastUserAction
      self.lastUserActionTime = lastUserActionTime
      self.lockStartTime = lockStartTime
      self.lockEndTime = lockEndTime
      self.userBoostedShare = 0
      self.locked = locked
      self.lockedAmount = lockedAmount
    
    def getAll(self):
      return {
          "shares": self.shares,
          "lastDepositedTime": self.lastDepositedTime,
          "tokenAtLastUserAction": self.tokenAtLastUserAction,
          "lastUserActionTime": self.lastUserActionTime,
          "lockStartTime": self.lockStartTime,
          "lockEndTime": self.lockEndTime,
          "userBoostedShare": self.userBoostedShare,
          "locked": self.locked,
          "lockedAmount": self.lockedAmount
      }


MAX_PERFORMANCE_FEE = 2000 # 20%
MAX_WITHDRAW_FEE = 500 # 5%
MAX_OVERDUE_FEE = 100 * 1e10 # 100%
MAX_WITHDRAW_FEE_PERIOD = 7 *60*60*24 # 1 week
# MIN_LOCK_DURATION = 7 *60*60*24
MIN_LOCK_DURATION = 3 *60
MAX_LOCK_DURATION_LIMIT = 1000 * 60*60*24
BOOST_WEIGHT_LIMIT = 5000 * 1e10
PRECISION_FACTOR = 1e12
PRECISION_FACTOR_SHARE = 1e28;
MIN_DEPOSIT_AMOUNT = 0.00001 *1e18
MIN_WITHDRAW_AMOUNT = 0.00001 *1e18

MAX_LOCK_DURATION = 365 *60*60*24
DURATION_FACTOR = 365 *60*60*24
BOOST_WEIGHT = 100 * 1e10

# UNLOCK_FREE_DURATION = 7 *60*60*24
UNLOCK_FREE_DURATION = 3 *60
DURATION_FACTOR_OVERDUE = 180 *60*60*24

performanceFee = 200  # 2%
performanceFeeContract = 200  # 2%
withdrawFee = 10 # 0.1%
withdrawFeeContract = 10  # 0.1%
overdueFee = 100 * 1e10   # 100%
withdrawFeePeriod = 72 *60*60  # 3 days


class BoostPool(object):

    def __init__(self, rewardDistributor: StakingRewards) -> None:
        self.address = 'boostPool'
        self.balanceOfToken = 0

        self.userInfo = {"address_0": UserInfo(), "alice": UserInfo(), "bob": UserInfo(), "kite": UserInfo()}

        self.rewardDistributor = rewardDistributor
        self.totalShares = 0
        self.totalBoostDebt = 0
        self.totalLockedAmount = 0

        self.harvestRewards = 0
      
    def deposit(self, amount, lockDuration, to):
        user = self.userInfo[to]
        totalLockDuration = lockDuration
        if user.lockEndTime >= self.blockTimestamp():
            if amount > 0:
                user.lockStartTime = self.blockTimestamp()
                self.totalLockedAmount -= user.lockedAmount
                user.lockedAmount = 0
            totalLockDuration += user.lockEndTime - user.lockStartTime
        assert lockDuration == 0 or totalLockDuration >= MIN_LOCK_DURATION, "Minimum lock period is one week"
        assert totalLockDuration <= MAX_LOCK_DURATION, "Maximum lock period exceeded"

        self._harvest()

        # if self.totalShares == 0:
        #     stockAmount = self.available()
        
        self._updateUserShare(to)

        if lockDuration > 0:
            if user.lockEndTime < self.blockTimestamp():
                user.lockStartTime = self.blockTimestamp()
                user.lockEndTime = self.blockTimestamp() + lockDuration
            else:
                user.lockEndTime += lockDuration
            user.locked = True
        
        currentShares = 0
        currentAmount = 0
        userCurrentLockedBalance = 0
        pool = self._balanceOf()
        if amount > 0:
            self.balanceOfToken += amount
            currentAmount = amount
        if user.shares > 0 and user.locked:
            userCurrentLockedBalance = (pool * user.shares) / self.totalShares
            currentAmount += userCurrentLockedBalance
            self.totalShares += user.shares
            user.shares = 0
            if user.lockStartTime == self.blockTimestamp():
                user.lockedAmount = userCurrentLockedBalance
                self.totalLockedAmount + user.lockedAmount
        if self.totalShares != 0:
            currentShares = (currentAmount * self.totalShares) / (pool - userCurrentLockedBalance)
        else:
            currentShares = currentAmount
        
        extBoostWeightOfUser = 1

        if user.lockEndTime > user.lockStartTime:
            boostWeight = ((user.lockEndTime - user.lockStartTime) * BOOST_WEIGHT)/DURATION_FACTOR
            boostShares = (boostWeight * currentShares)/PRECISION_FACTOR
            currentShares += boostShares

            extBoostShares = (extBoostWeightOfUser * currentShares) / PRECISION_FACTOR
            currentShares += extBoostShares
            user.shares += currentShares

            userBoostedShare = currentAmount * ((boostWeight + extBoostWeightOfUser) * PRECISION_FACTOR + boostWeight * extBoostShares) / (PRECISION_FACTOR * PRECISION_FACTOR)
            user.userBoostedShare += userBoostedShare
            self.totalBoostDebt += userBoostedShare

            user.lockedAmount += amount
            self.totalLockedAmount += amount
            print('==> lock', to, user.lockedAmount, user.shares)
        else:
            extBoostShares = (extBoostWeightOfUser * currentShares) / PRECISION_FACTOR
            print("not locking, extBoostShares", extBoostShares)
            currentShares += extBoostShares
            user.shares += currentShares
            userBoostedShare = (extBoostWeightOfUser * currentAmount) / PRECISION_FACTOR
            user.userBoostedShare += userBoostedShare
            self.totalBoostDebt += userBoostedShare
        
        if amount > 0 or lockDuration > 0:
            user.lastDepositedTime = self.blockTimestamp()
          
        self.totalShares += currentShares

        user.tokenAtLastUserAction = (user.shares * self.balanceOfToken) / self.totalShares - user.userBoostedShare
        user.lastUserActionTime = self.blockTimestamp()

    def withdraw(self, shares, amount, to):
        user = self.userInfo[to]
        assert shares <= user.shares, "Withdraw amount exceeds balance"
        assert user.lockEndTime < self.blockTimestamp(), "Still in lock"

        currentShares = shares
        sharesPercent = shares * PRECISION_FACTOR_SHARE / user.shares

        self._harvest()

        self._updateUserShare(to)

        if shares == 0 and amount > 0:
            pool = self._balanceOf()
            currentShares = amount * self.totalShares / pool
            if currentShares > user.shares:
                currentShares = user.shares
        else:
            currentShares = sharesPercent * user.shares / PRECISION_FACTOR_SHARE
        
        outAmount = self._balanceOf() * user.shares / self.totalShares
        self.balanceOfToken -= outAmount

        user.shares -= currentShares
        self.totalShares -= currentShares

        if user.shares > 0:
            currentAmount = self._balanceOf() * user.shares / self.totalShares
            extBoostWeightOfUser = 1
            extBoostShares = (extBoostWeightOfUser * user.shares) / PRECISION_FACTOR
            user.shares += extBoostShares
            self.totalShares += extBoostShares
            userBoostedShare = (extBoostWeightOfUser * currentAmount) / PRECISION_FACTOR
            user.userBoostedShare += userBoostedShare
            self.totalBoostDebt += userBoostedShare
            

            user.tokenAtLastUserAction = user.shares * self._balanceOf() / self.totalShares - user.userBoostedShare
        else:
            user.tokenAtLastUserAction = 0
        
        user.lastUserActionTime = self.blockTimestamp()

        return outAmount

    def unlock(self, to):
        user = self.userInfo[to]
        assert user.locked == True, "Not locked"

        self.deposit(0, 0, to)
      
    def _harvest(self):
        self.harvestRewards += self.rewardDistributor.getReward(self.address)
    
    def _updateUserShare(self, to):
        user = self.userInfo[to]
        if user.shares > 0:
            if user.userBoostedShare > 0:
                currentAmount = (self.balanceOfToken * user.shares) / self.totalShares - user.userBoostedShare
                self.totalBoostDebt -= user.userBoostedShare
                user.userBoostedShare = 0
                self.totalShares -= user.shares
                
                if user.locked and (user.lockEndTime + UNLOCK_FREE_DURATION) < self.blockTimestamp():
                    earnAmount = currentAmount - user.lockedAmount
                    overdueDuration = self.blockTimestamp() - user.lockEndTime - UNLOCK_FREE_DURATION
                    if overdueDuration > DURATION_FACTOR_OVERDUE:
                        overdueDuration = DURATION_FACTOR_OVERDUE
                    
                    overdueWeight = (overdueDuration * overdueFee) / DURATION_FACTOR_OVERDUE
                    currentOverdueFee = (earnAmount * overdueWeight) / PRECISION_FACTOR
                    # token.safeTransfer(treasury, currentOverdueFee)
                    currentAmount -= currentOverdueFee

                pool = self._balanceOf()
                currentShares = 0
                if self.totalShares != 0:
                    currentShares = (currentAmount * self.totalShares) / (pool - currentAmount)
                else:
                    currentShares = currentAmount
                user.shares = currentShares
                self.totalShares += currentShares

                if user.locked and user.lockEndTime < self.blockTimestamp():
                    user.locked = False
                    user.lockStartTime = 0
                    user.lockEndTime = 0
                    self.totalLockedAmount -= user.lockedAmount
                    user.lockedAmount = 0
                    print('==> unlock', to, currentAmount)
    
    def _balanceOf(self):
        return self.balanceOfToken + self.totalBoostDebt
        
    
    def blockTimestamp(self):
        """Akin to `block.timestamp`"""
        return calendar.timegm(time.gmtime())
    

Init

In [51]:
rewardDistributor = StakingRewards()

pool = BoostPool(rewardDistributor)

### Staking without Lock

In [12]:
# Alice staking

alice = "alice"
stakingAmount = 100e18

pool.deposit(stakingAmount, 0, alice)

print("user infos", pool.userInfo[alice].getAll())

print("pool balanceOfToken", pool.balanceOfToken)
print("pool totalShares", pool.totalShares)
print("pool totalBoostDebt", pool.totalBoostDebt)
print("pool totalLockedAmount", pool.totalLockedAmount)
print("pool harvestRewards", pool.harvestRewards)

not locking, extBoostShares 100000000.0
user infos {'shares': 1.000000000001e+20, 'lastDepositedTime': 1675306759, 'tokenAtLastUserAction': 9.99999999999e+19, 'lastUserActionTime': 1675306759, 'lockStartTime': 0, 'lockEndTime': 0, 'userBoostedShare': 100000000.0, 'locked': False, 'lockedAmount': 0}
pool balanceOfToken 1e+20
pool totalShares 1.000000000001e+20
pool totalBoostDebt 100000000.0
pool totalLockedAmount 0
pool harvestRewards 0.0


In [13]:
# Alice staking again

stakingAmount2 = 50e18

pool.deposit(stakingAmount2, 0, alice)

print("T2 user infos", pool.userInfo[alice].getAll())

print("pool balanceOfToken", pool.balanceOfToken)
print("pool totalShares", pool.totalShares)
print("pool totalBoostDebt", pool.totalBoostDebt)
print("pool totalLockedAmount", pool.totalLockedAmount)
print("pool harvestRewards", pool.harvestRewards)

not locking, extBoostShares 49999999.99995
T2 user infos {'shares': 1.499999999999e+20, 'lastDepositedTime': 1675306763, 'tokenAtLastUserAction': 1.4999999999995e+20, 'lastUserActionTime': 1675306763, 'lockStartTime': 0, 'lockEndTime': 0, 'userBoostedShare': 50000000.0, 'locked': False, 'lockedAmount': 0}
pool balanceOfToken 1.5e+20
pool totalShares 1.499999999999e+20
pool totalBoostDebt 50000000.0
pool totalLockedAmount 0
pool harvestRewards 0.0


In [14]:
# Alice withdraw

shares = 1.499999999999e+20

outAmount = pool.withdraw(shares, 0, alice)
print("withdraw token amount", outAmount)
print("after withdraw user infos", pool.userInfo[alice].getAll())

print("pool balanceOfToken", pool.balanceOfToken)
print("pool totalShares", pool.totalShares)
print("pool totalBoostDebt", pool.totalBoostDebt)
print("pool totalLockedAmount", pool.totalLockedAmount)
print("pool harvestRewards", pool.harvestRewards)


withdraw token amount 1.5e+20
after withdraw user infos {'shares': 0.0, 'lastDepositedTime': 1675306763, 'tokenAtLastUserAction': 0, 'lastUserActionTime': 1675306772, 'lockStartTime': 0, 'lockEndTime': 0, 'userBoostedShare': 0, 'locked': False, 'lockedAmount': 0}
pool balanceOfToken 0.0
pool totalShares 0.0
pool totalBoostDebt 0.0
pool totalLockedAmount 0
pool harvestRewards 0.0


### Staking with lock

In [38]:
# Alice staking

alice = "alice"
stakingAmount = 100e18

pool.deposit(stakingAmount, MIN_LOCK_DURATION, alice)

print("user infos", pool.userInfo[alice].getAll())

print("pool balanceOfToken", pool.balanceOfToken)
print("pool totalShares", pool.totalShares)
print("pool totalBoostDebt", pool.totalBoostDebt)
print("pool totalLockedAmount", pool.totalLockedAmount)
print("pool harvestRewards", pool.harvestRewards)

==> lock alice 1e+20 1.0000057077635572e+20
user infos {'shares': 1.0000057077635572e+20, 'lastDepositedTime': 1675308224, 'tokenAtLastUserAction': 9.999942916656634e+19, 'lastUserActionTime': 1675308224, 'lockStartTime': 1675308224, 'lockEndTime': 1675308404, 'userBoostedShare': 570833433659118.9, 'locked': True, 'lockedAmount': 1e+20}
pool balanceOfToken 1e+20
pool totalShares 1.0000057077635572e+20
pool totalBoostDebt 570833433659118.9
pool totalLockedAmount 1e+20
pool harvestRewards 0.0


In [48]:
# after MIN_LOCK_DURATION to withdraw
unlockMinDate = datetime.fromtimestamp(pool.userInfo[alice].getAll()['lastDepositedTime']) + timedelta(seconds=MIN_LOCK_DURATION)
print("min unlock datetime", unlockMinDate)

shares = 1.0000057077635572e+20

outAmount = pool.withdraw(shares, 0, alice)
print("unlock and withdraw token amount", outAmount)
print("after withdraw user infos", pool.userInfo[alice].getAll())

print("pool balanceOfToken", pool.balanceOfToken)
print("pool totalShares", pool.totalShares)
print("pool totalBoostDebt", pool.totalBoostDebt)
print("pool totalLockedAmount", pool.totalLockedAmount)
print("pool harvestRewards", pool.harvestRewards)



min unlock datetime 2023-02-02 03:26:44
==> unlock alice 9.999942916656634e+19
unlock and withdraw token amount 1e+20
after withdraw user infos {'shares': 0.0, 'lastDepositedTime': 1675308224, 'tokenAtLastUserAction': 0, 'lastUserActionTime': 1675308411, 'lockStartTime': 0, 'lockEndTime': 0, 'userBoostedShare': 0, 'locked': False, 'lockedAmount': 0}
pool balanceOfToken 0.0
pool totalShares 0.0
pool totalBoostDebt 0.0
pool totalLockedAmount 0.0
pool harvestRewards 0.0


In [47]:
datetime.now()

datetime.datetime(2023, 2, 2, 3, 26, 46, 349463)